In [136]:
pwd

'/Users/cohlem/Projects/OkProfessor/researcher/notebooks'

In [137]:
import sys
sys.path.append('/Users/cohlem/Projects/OkProfessor/')

In [138]:
from researcher.config import Config
from researcher.search.duckduckgo import Duckduckgo
from researcher.utils.functions import * 


class Researcher:
    def __init__(self, query ):
        self.query = query
        self.cfg = Config()
        self.agent = None
        self.role = None
        self.visited_urls = set()
        self.context = []
        
    async def run(self):
        """
        Run the researcher
        """
        if self.cfg.search_engine == 'Duckduckgo':
            retriever = Duckduckgo()
            
        print(f'Running research for query: {self.query}')
        self.agent, self.role = await choose_agent(self.query, self.cfg.llm )
        print(f'Running {self.agent} ...')
        
        #query modification
        sub_queries = await get_sub_queries(self.query, self.role, self.cfg) + [self.query]
        
        for each_query in sub_queries:
            
            print(f'Running research for query {each_query}')
            content = await get_content_using_query(each_query)
            
            context = await get_similar_context(each_query, content)
            
            self.context.append(context)
        
        
        
        

        
    async def get_content_using_query(query):

        search_engine = Duckduckgo(query = query)
        search_urls = search_engine.search(max_results = r.cfg.max_search_results_per_query)

        search_urls = [url.get('href') for url in search_urls]

        new_search_urls = get_unique_urls(search_urls) #filter out the same urls 
       

        content_scraper = Scraper(new_search_urls)
        content = content_scraper.run()

        return content
    
    
    async def get_unique_urls(self, urls):
        
        new_urls = []
        for url in urls:
            if url not in self.visited_urls:
                
                print(f'✅ Adding url {url} to our research')
                
                new_urls.append(url)
                self.visited_urls.add(url)
                
        return new_urls
                
    
    async def get_similar_context(self, query, content):
        pass

In [78]:
r = Researcher('investment in real estate and buying properties')

In [7]:
 
r = Researcher('investment in real estate and buying properties')

response = await r.run()

print(response)



Running research for query: investment in real estate and buying properties
Running 🏢 Real Estate Agent ...
['Real estate market trends November 2023', 'Property value forecast 2024', 'Rental income potential in [specific location]', 'investment in real estate and buying properties']


In [23]:
response

['Real estate market trends November 2023',
 'Property value forecast 2024',
 'Rental income potential in [specific location]',
 'investment in real estate and buying properties']

In [10]:
from researcher.search.duckduckgo import Duckduckgo

In [139]:
search_engine = Duckduckgo('investment in real estate and buying properties')


In [140]:
search_urls = list(search_engine.search(max_results=r.cfg.max_search_results_per_query))

In [141]:
search_urls = [url['href'] for url in search_urls]

In [142]:
search_urls

['https://www.businessinsider.com/how-to-find-good-deal-property-buy-real-estate-investing-2023-11?op=1',
 'https://www.investopedia.com/mortgage/real-estate-investing-guide/',
 'https://smartasset.com/investing/how-to-buy-your-first-investment-property',
 'https://time.com/personal-finance/article/how-to-invest-in-real-estate/',
 'https://money.usnews.com/investing/real-estate-investments/articles/rules-for-buy-and-hold-real-estate-investing']

In [27]:
response

['Real estate market trends November 2023',
 'Property value forecast 2024',
 'Rental income potential in [specific location]',
 'investment in real estate and buying properties']

In [56]:
from researcher.scraping.scrape import Scraper

In [ ]:
def get_unique_urls(urls):
    self.visited_urls 

In [150]:
async def get_content_using_query(query):
    
    search_engine = Duckduckgo(query = query)
    search_urls = search_engine.search(max_results = r.cfg.max_search_results_per_query)
    
    search_urls = [url.get('href') for url in search_urls]
    
#     new_search_urls = get_unique_urls(search_urls)
#     print(search_urls)
    
    scraper = Scraper(search_urls)
    content = scraper.run()
    
    
    
    return content
    
    

In [151]:
ans = await get_content_using_query('Property value forecast 2024')

/Users/cohlem/anaconda3/envs/okprofessor/lib/python3.9/site-packages/bs4/element.py:1277: RuntimeWarning: coroutine 'run_async_tasks.<locals>._gather' was never awaited
  self.setup(parent, previous)
/Users/cohlem/anaconda3/envs/okprofessor/lib/python3.9/site-packages/bs4/element.py:1277: RuntimeWarning: coroutine 'BaseRetriever.aretrieve' was never awaited
  self.setup(parent, previous)


In [152]:
ans

[{'url': 'https://www.realtor.com/research/2024-national-housing-forecast/',
  'raw_content': 'Articles\n2024 Housing Market Forecast and Predictions: Housing Affordability Finally Begins to Turnaround\nAs we look ahead to 2024, we see a mix of continuity and change in both the housing market and economy. Against a backdrop of modest economic growth, slightly higher unemployment, and easing inflation longer term interest rates including mortgage rates begin a slow retreat. The shift from climbing to falling mortgage rates improves housing affordability, but saps some of the urgency home shoppers had previously sensed. Less frenzied housing demand and plenty of rental home options keep home sales relatively stable at low levels in 2024, helping home prices to adjust slightly lower even as the number of for-sale homes continues to dwindle.\nRealtor.com® 2024 Forecast for Key Housing Indicators\nHome Prices Dip, Improving Affordability\nHome prices grew at a double-digit annual clip for t

In [53]:


for each_query in response:
    
    content = await get_content_using_query(each_query)
    
    relevant_context = 
    self.context.append() #all relevant context
    
    

In [41]:
len(results)

5

In [25]:
search_urls

{'https://smartasset.com/investing/how-to-buy-your-first-investment-property',
 'https://time.com/personal-finance/article/how-to-invest-in-real-estate/',
 'https://www.businessinsider.com/how-to-find-good-deal-property-buy-real-estate-investing-2023-11?op=1',
 'https://www.forbes.com/sites/forbesbusinesscouncil/2023/10/30/exploring-the-pros-and-cons-of-real-estate-investment/',
 'https://www.investopedia.com/mortgage/real-estate-investing-guide/'}

In [29]:
gg = {'a': 1, 'b': 2}

In [32]:
ans = gg.get('c')

## Langchain's hybrid search

In [133]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings

In [134]:

# initialize the bm25 retriever and faiss retriever
bm25_retriever = BM25Retriever.from_texts(context)
bm25_retriever.k = 2

embedding = OpenAIEmbeddings()
faiss_vectorstore = FAISS.from_texts(context, embedding) 

AttributeError: 'Document' object has no attribute 'split'

In [103]:


# initialize the bm25 retriever and faiss retriever
bm25_retriever = BM25Retriever.from_texts(context)
bm25_retriever.k = 2

embedding = OpenAIEmbeddings()
faiss_vectorstore = FAISS.from_texts(context, embedding)


faiss_retriever = faiss_vectorstore.as_retriever(search_kwargs={"k": 2})



# initialize the ensemble retriever
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, faiss_retriever], weights=[0.5, 0.5]
)

In [107]:
docs = ensemble_retriever.get_relevant_documents("i very much love apples")
docs

[Document(page_content='I like apples'),
 Document(page_content='I like oranges'),
 Document(page_content='Apples and oranges are fruits')]

In [106]:
faiss_retriever.get_relevant_documents('i very much love apples')

[Document(page_content='I like apples'),
 Document(page_content='I like oranges')]

In [166]:
import os
from enum import Enum
from typing import Any, Dict, List, Optional

from langchain.callbacks.manager import CallbackManagerForRetrieverRun
from langchain.schema import Document
from langchain.schema.retriever import BaseRetriever


class SearchAPIRetriever(BaseRetriever):
    """Search API retriever."""
    pages: List[Dict] = []

    def _get_relevant_documents(
        self, query: str, *, run_manager: CallbackManagerForRetrieverRun
    ) -> List[Document]:

        docs = [
            Document(
                page_content=page.get("raw_content", ""),
                metadata={
#                     "title": page.get("title", ""),
                    "source": page.get("url", ""),
                },
            )
            for page in self.pages
        ]

        return docs

In [116]:
base_retriever = SearchAPIRetriever(pages = ans)

In [119]:
from researcher.context.chunking import Chunking

In [120]:
chunking = Chunking(1000,100)

In [122]:
len(ans)

5

In [153]:
ans

[{'url': 'https://www.realtor.com/research/2024-national-housing-forecast/',
  'raw_content': 'Articles\n2024 Housing Market Forecast and Predictions: Housing Affordability Finally Begins to Turnaround\nAs we look ahead to 2024, we see a mix of continuity and change in both the housing market and economy. Against a backdrop of modest economic growth, slightly higher unemployment, and easing inflation longer term interest rates including mortgage rates begin a slow retreat. The shift from climbing to falling mortgage rates improves housing affordability, but saps some of the urgency home shoppers had previously sensed. Less frenzied housing demand and plenty of rental home options keep home sales relatively stable at low levels in 2024, helping home prices to adjust slightly lower even as the number of for-sale homes continues to dwindle.\nRealtor.com® 2024 Forecast for Key Housing Indicators\nHome Prices Dip, Improving Affordability\nHome prices grew at a double-digit annual clip for t

In [198]:
sub_query_chunks = []
for each_ans in ans:
    sub_query_chunks += chunking.run(content=each_ans['raw_content'], metadatas= {'url': each_ans['url'] })

In [199]:
len(sub_query_chunks)

49

In [159]:
ans

[{'url': 'https://www.realtor.com/research/2024-national-housing-forecast/',
  'raw_content': 'Articles\n2024 Housing Market Forecast and Predictions: Housing Affordability Finally Begins to Turnaround\nAs we look ahead to 2024, we see a mix of continuity and change in both the housing market and economy. Against a backdrop of modest economic growth, slightly higher unemployment, and easing inflation longer term interest rates including mortgage rates begin a slow retreat. The shift from climbing to falling mortgage rates improves housing affordability, but saps some of the urgency home shoppers had previously sensed. Less frenzied housing demand and plenty of rental home options keep home sales relatively stable at low levels in 2024, helping home prices to adjust slightly lower even as the number of for-sale homes continues to dwindle.\nRealtor.com® 2024 Forecast for Key Housing Indicators\nHome Prices Dip, Improving Affordability\nHome prices grew at a double-digit annual clip for t

In [197]:
def get_similar_context(query, content):
    chunks = []
    chunking = Chunking(1000,100)
    
    for each_content in content:
        chunks += chunking.run(content=each_content['raw_content'], metadatas= {'url': each_content['url'] })
   
    retriever = hybrid_retriever(chunks,8)
    
    relevant_chunks = retriever.get_relevant_documents(query)
    
    return relevant_chunks

In [191]:
final_gg = get_similar_context('investment in 2024', ans)

In [192]:
len(final_gg)

12

In [193]:
final_gg

[Document(page_content='Many companies featured on Money advertise with us. Opinions are our own, but compensation and in-depth research may determine where and how companies appear. Learn more about how we make money.\nHere’s How Far Home Prices and Mortgage Rates Could Drop in 2024\nhttps://money.com/mortgage-rates-home-prices-predictions-2024/\nThe new year could finally bring good tidings for homebuyers — or at least the beginning of improved housing affordability.\nHistorically high mortgage rates and housing prices that stymied homebuyers this year are expected to ebb a bit in 2024, according to real estate brokerage Realtor.com’s new housing market forecast. While home shoppers shouldn’t expect major relief from today’s crushing homeownership costs, small gains in affordability are expected to help some buyers get a foot in the door.\nRenters can also look forward to somewhat better conditions as new construction hits the market in the coming months.', metadata={'url': 'https://

In [194]:
class HybridRetriever:
    def __init__(self, documents, embeddings=OpenAIEmbeddings(), max_results=5):

        self.documents = documents
        self.embeddings = embeddings
        self.max_results = max_results
        self.retriever = self._hybrid_retriever()
        
    def _hybrid_retriever(self):
        
        # initialize the bm25 retriever and faiss retriever
        bm25_retriever = BM25Retriever.from_documents(self.documents)
        bm25_retriever.k = self.max_results

        embedding = OpenAIEmbeddings()
        faiss_vectorstore = FAISS.from_documents(self.documents, self.embeddings)
        faiss_retriever = faiss_vectorstore.as_retriever(search_kwargs={"k": self.max_results })

        #initialize the ensemble retriever
        ensemble_retriever = EnsembleRetriever(
            retrievers=[bm25_retriever, faiss_retriever], weights=[0.2, 0.8]
        )

        return ensemble_retriever
    
    def get_context(self, query):

        return self.retriever.get_relevant_documents(query)
        
        

In [ ]:
H = HybridRetriever()

In [196]:
len(ans)

4

In [181]:
def hybrid_retriever(context, K):
    
    
    # initialize the bm25 retriever and faiss retriever
    bm25_retriever = BM25Retriever.from_documents(context)
    bm25_retriever.k = K

    embedding = OpenAIEmbeddings()
    faiss_vectorstore = FAISS.from_documents(context, embedding)

    faiss_retriever = faiss_vectorstore.as_retriever(search_kwargs={"k": K})



    #initialize the ensemble retriever
    ensemble_retriever = EnsembleRetriever(
        retrievers=[bm25_retriever, faiss_retriever], weights=[0.2, 0.8]
    )
    
    return ensemble_retriever

In [182]:
retriever = hybrid_retriever(final_gg, 5)

In [184]:
combined_ans = retriever.get_relevant_documents('investments')

In [186]:
len(combined_ans)

8

In [170]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.vectorstores import FAISS

doc_list = [
    "I like apples",
    "I like oranges",
    "Apples and oranges are fruits",
]

# initialize the bm25 retriever and faiss retriever
bm25_retriever = BM25Retriever.from_texts(doc_list)
bm25_retriever.k = 2

embedding = OpenAIEmbeddings()
faiss_vectorstore = FAISS.from_texts(doc_list, embedding)
faiss_retriever = faiss_vectorstore.as_retriever(search_kwargs={"k": 2})

# initialize the ensemble retriever
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, faiss_retriever], weights=[0.5, 0.5]
)

docs = ensemble_retriever.get_relevant_documents("apples")
docs

[Document(page_content='I like apples'),
 Document(page_content='Apples and oranges are fruits')]

In [171]:
bm25_retriever.get_relevant_documents('apples')

[Document(page_content='I like apples'),
 Document(page_content='Apples and oranges are fruits')]